In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import pandas as pd
#!wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
#!pip install scikit-learn

import sklearn
'''from google.colab import files
src = list(files.upload().values())[0]
open('.py','wb').write(src)
import mylib'''
import io
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import math
import copy

In [3]:
data = pd.read_csv('train_data_QUICK_START.csv')
sequences = data['sequence']
reactivity = data.iloc[:,4:-1]

False

In [4]:
#update NaN to csv file
m = reactivity.mean(axis=1)
for i, col in enumerate(reactivity):
    # using i allows for duplicate co lumns
    # inplace *may* not always work here, so IMO the next line is preferred
    # df.iloc[:, i].fillna(m, inplace=True)
    reactivity.iloc[:, i] = reactivity.iloc[:, i].fillna(m)
reactivity.to_csv('ReactValues',sep=',', index=False, encoding='utf-8')

In [15]:
reactivity = pd.read_csv('ReactValues')

In [5]:
reactivity.to_csv('ReactValues',sep=',', index=False, encoding='utf-8')



In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(sequences, reactivity, test_size=0.2, random_state=42)

def encode_sequence(sequence):
    encoding = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'U': [0, 0, 0, 1]}
    encoded_sequence = []
    for nucleotide in sequence:
        # Check if the nucleotide is in the encoding dictionary
        if nucleotide in encoding:
            encoded_sequence.append(encoding[nucleotide])
        else:
            # Handle unexpected characters (e.g., print a warning)
            print(f"Warning: Unexpected character '{nucleotide}' in sequence.")
            # You can choose to replace the unexpected character with a default encoding or ignore it

    # Convert the list of encoded nucleotides to a PyTorch tensor
    return torch.FloatTensor(encoded_sequence)

from torch.nn.utils.rnn import pad_sequence

# Encode sequences
X_train_encoded = [encode_sequence(seq) for seq in X_train]
X_valid_encoded = [encode_sequence(seq) for seq in X_valid]

# Pad sequences to make them equal length
X_train_padded = torch.tensor(pad_sequence(X_train_encoded, batch_first=True))
X_valid_padded = torch.tensor(pad_sequence(X_valid_encoded, batch_first=True))

C:\Users\matth\AppData\Local\Temp\ipykernel_24984\1744887545.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_padded = torch.tensor(pad_sequence(X_train_encoded, batch_first=True))
C:\Users\matth\AppData\Local\Temp\ipykernel_24984\1744887545.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_valid_padded = torch.tensor(pad_sequence(X_valid_encoded, batch_first=True))


In [6]:
total = pd.DataFrame(X_train_padded)

NameError: name 'X_train_padded' is not defined

In [22]:
print(X_train_padded.shape)
print(X_valid_padded.shape)
print(X_train.shape)

torch.Size([268492, 206, 4])
torch.Size([67124, 206, 4])
(268492,)


In [ ]:
class customImageDataset(Dataset):
    def __init__(self, ): 

In [1]:
# MultiHeadAttention is the process of giving attention to different parts of a sequence simultaneously to find which parts of a 
# sequence affect each other.
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        # the model handles the queries, keys, and values automatically
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

NameError: name 'nn' is not defined

In [9]:
#Creates a network for passing information forward through the model using two linear layers and a non-linear activation function
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        #2 linear and one nonlinear layer
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()
#overwrites the nn.Module forward function to pass the data through the model
    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [6]:
#allows the model to understand the position of objects relative to a sequence
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        #creates an empty tensor
        pe = torch.zeros(max_seq_length, d_model)
        #math stuff I don't need to fully understand
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        #uses a sin and cosine function with different frequencies to "plot the points"
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [9]:
# Encoder layer takes and processes the inputs
# Consists of a MultiHeadAttention, PositionWiseFeedForward, two normalization layers, and
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        #Creates the attention layer and feed forward network for this encoder layer
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        #Two normalization layers to prevent overfitting of hidden layers during processing
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        # turns some input sequences to 0 to help prevent overfitting. The var dropout is the probability that this will happen, usually set between 0.2-0.5
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        #Don't understand why x is inputted three times
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [10]:
# The decoder passes the data from the encoder through two layers of multihead attention layers (the first one masked) 
# and then a position wise feed forward layer
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        # method of passing the information forward through the normalization layers and cross and self attention layers
        # May want to read more on the different kinds of multi-head-attention layers
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [27]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        # The purpose of nn.embedding is to take discrete tokens, like words, and change them into more vectors that can be operated on. However,
        # we already changed the sequence of DNA to orthogonal vectors, so does that mean I should just take this out? (learn more)
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        #gives information about the position to the model
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        #Creates the same amount of encoder and decoder layers based on the number of layers user specifies
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)
# The purpose of the mask is to hide some of the parts of the sequence from each other to prevent the 
# model from gaining information on future or irrelevant parts of the sequence
    def generate_mask(self, src, tgt):
        #The model creates the mask automatically, I don't need to worry about the specifics
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output